# Vorhersage des Modelergebnisses auf Grundlage der Bewertungsfunktion

Im Folgenden wird untersucht, wie gut das entwickelte Modell das eigene Abschneiden bewertet mit Hilfe der Bewertungsfunktion abschätzen kann.

Getestet wird nur das Klassifikationsmodell, als Schaden eines FRAUD-Falles wird der Mittelwert über alle Transaktionen genommen.

Ergebnis: die Methode ist ungeeignet, da sich auch über sehr kleine Wahrscheinlichkeiten multipliziert mit dem Mittelwert des Schadens hohe Kosten ergeben, die in Summe viel höher sind als die tatsächlichen Kosten. Das Modell ist also nicht in der Lage, die Kostenfunktion zu approximieren.

In [60]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split
from xgboost import XGBClassifier

import fraud_detection as fd
from fraud_detection import data_loader, metrics
from fraud_detection.models.costoptim import bewertung

datapath = "../data/transformed_label_and_damage.parquet"
seed = 42

In [51]:
seed = 42

In [52]:
# lade Daten ohne die nutzlosen Features
X, targets = data_loader.load_data_np(datapath, drop_features=data_loader.useless_features)

X_train, X_test, y_train, y_test = train_test_split(
    X, targets, test_size=0.2, random_state=seed, stratify=targets[:, 1] > 0
)

In [56]:
def estimate_bewertung(preds, damages, gain_tp=5, cost_fp=10):
    """Estimate the bewertung value for given predictions and damages."""
    cost_control = preds * gain_tp - (1 - preds) * cost_fp
    cost_no_control = -(1 - preds) * damages
    costs = np.vstack([cost_control, cost_no_control])
    print(costs)
    costs = costs.max(axis=0)
    print(costs)
    return costs.sum()

In [57]:
clf = XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    objective="binary:logistic",
)
clf.fit(X_train, y_train[:, 0])

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [59]:
probs = clf.predict_proba(X_test)[:, 1]
preds = clf.predict(X_test)
idx_damages_only = np.where(y_test[:, 1] > 0)[0]
mean_damage = y_test[idx_damages_only, 1].mean()
#mean_damage = y_test[:, 1].mean()
damage = np.full_like(probs, mean_damage)

estimate = estimate_bewertung(probs, damage, gain_tp=5, cost_fp=10)

print(f"Das Modell schätzt, dass es insgesamt {estimate:.2f} Kosten einspart.")

[[-9.841874  -9.98262   -9.867     ... -9.908418  -9.90766   -9.948496 ]
 [-6.8914614 -6.9568133 -6.9031277 ... -6.9223595 -6.922007  -6.940969 ]]
[-6.8914614 -6.9568133 -6.9031277 ... -6.9223595 -6.922007  -6.940969 ]
Das Modell schätzt, dass es insgesamt -197509.50 Kosten einspart.


Tatsächliche Bewertung:

In [48]:
#
ms = metrics.bewertung(probs, preds, y_test[:, 0], y_test[:, 1],)
metrics.print_metrics(ms)

cm
[[28585    89]
 [  384   472]]
precision                0.84
recall                   0.55
f1                       0.67
mcc                      0.67
auc-pr:                  0.73
damage_total          5961.94
damage_prevented      3689.64
damage_missed         2272.30
detected bonus        2360.00
fp penalty             890.00
Bewertung             -802.30


In [37]:
mean_damage

np.float64(6.964883177570094)